In [1]:
import pandas as pd

In [2]:
def stringify_query(query: dict[str, str]) -> str:
    return " & ".join([f"{key}=='{value}'" for key, value in query.items()])

In [3]:
dictionary_path = "/Users/joselondono/Downloads/Dictionary.xlsx"
vars = pd.read_excel(dictionary_path)

### Renter costs

In [4]:
def parse_rent_groups(s: pd.Series) -> pd.Series:
    return (
        s
        .str.replace("$", "", regex=False)
        .str.replace(" to " , "-", regex=False)
        .str.replace(" or more", "-", regex=False)
        .str.replace("Less than ", "0-", regex=False)
        .str.replace(" ", "")
    )
    
def get_numeric_rent(s: pd.Series) -> pd.Series:
    return s.str.split("-", expand=True)[0].astype(int)

In [5]:
rent_query = {"TABLE_TITLE": "Gross Rent", "FIELD_LEVEL_1": "Estimate"}
rent_vars = vars.query(stringify_query(rent_query))
non_empty_rent_vars = (
    rent_vars.query("FIELD_LEVEL_6.notna()")
    .assign(
        name=lambda df: parse_rent_groups(df["FIELD_LEVEL_6"]),
        numeric=lambda df: get_numeric_rent(df["name"])
    )
    .rename(columns={"TABLE_ID": "codes"})
    .groupby("FIELD_LEVEL_6")
    .agg({"codes": list, "name": "first", "numeric": "first"})
    .sort_values("numeric")
)

## Family Income

In [7]:
family_income_query = {"TABLE_NUMBER": "B19001", "FIELD_LEVEL_1": "Estimate"}
family_income_ids = vars.query(stringify_query(family_income_query)).query("FIELD_LEVEL_5.notna()")

In [9]:
(
    family_income_ids
    .assign(
        name=lambda df: parse_rent_groups(df["FIELD_LEVEL_5"]),
        numeric=lambda df: get_numeric_rent(df["name"])
    )
    .rename(columns={"TABLE_ID": "codes"})
    .groupby("FIELD_LEVEL_5")
    .agg({"codes": list, "name": "first", "numeric": "first"})
    .sort_values("numeric")
    .to_dict("index")
)

{'Less than $10 000': {'codes': ['B19001e2'], 'name': '0-10000', 'numeric': 0},
 '$10 000 to $14 999': {'codes': ['B19001e3'],
  'name': '10000-14999',
  'numeric': 10000},
 '$15 000 to $19 999': {'codes': ['B19001e4'],
  'name': '15000-19999',
  'numeric': 15000},
 '$20 000 to $24 999': {'codes': ['B19001e5'],
  'name': '20000-24999',
  'numeric': 20000},
 '$25 000 to $29 999': {'codes': ['B19001e6'],
  'name': '25000-29999',
  'numeric': 25000},
 '$30 000 to $34 999': {'codes': ['B19001e7'],
  'name': '30000-34999',
  'numeric': 30000},
 '$35 000 to $39 999': {'codes': ['B19001e8'],
  'name': '35000-39999',
  'numeric': 35000},
 '$40 000 to $44 999': {'codes': ['B19001e9'],
  'name': '40000-44999',
  'numeric': 40000},
 '$45 000 to $49 999': {'codes': ['B19001e10'],
  'name': '45000-49999',
  'numeric': 45000},
 '$50 000 to $59 999': {'codes': ['B19001e11'],
  'name': '50000-59999',
  'numeric': 50000},
 '$60 000 to $74 999': {'codes': ['B19001e12'],
  'name': '60000-74999',
  'numer

## Enrolled in School

In [32]:
school_enrollment_query = {"TABLE_NUMBER": "B14007", "FIELD_LEVEL_1": "Estimate"}
enroll = vars.query(stringify_query(school_enrollment_query)).query("FIELD_LEVEL_6.notna()")

In [35]:
GRADES = {
    "Enrolled in kindergarten": "ELEMENTARY_SCHOOL",
    "Enrolled in grade 1": "ELEMENTARY_SCHOOL",
    "Enrolled in grade 2": "ELEMENTARY_SCHOOL",
    "Enrolled in grade 3": "ELEMENTARY_SCHOOL",
    "Enrolled in grade 4": "ELEMENTARY_SCHOOL",
    "Enrolled in grade 5": "ELEMENTARY_SCHOOL",
    "Enrolled in grade 6": "MIDDLE_SCHOOL",
    "Enrolled in grade 7": "MIDDLE_SCHOOL",
    "Enrolled in grade 8": "MIDDLE_SCHOOL",
    "Enrolled in grade 9": "HIGH_SCHOOL",
    "Enrolled in grade 10": "HIGH_SCHOOL",
    "Enrolled in grade 11": "HIGH_SCHOOL",
    "Enrolled in grade 12": "HIGH_SCHOOL",
    "Enrolled in nursery school preschool": "HIGHER_EDUCATION",
    "Enrolled in college undergraduate years": "HIGHER_EDUCATION",
    "Graduate or professional school": "HIGHER_EDUCATION",
}

In [36]:
(
    enroll.assign(name=lambda df: df["FIELD_LEVEL_6"].map(GRADES))
    .rename(columns={"TABLE_ID": "codes"})
    .groupby("name")
    .agg({"codes": list, "name": "first", })
    .to_dict("index")
)

{'ELEMENTARY_SCHOOL': {'codes': ['B14007e4',
   'B14007e5',
   'B14007e6',
   'B14007e7',
   'B14007e8',
   'B14007e9'],
  'name': 'ELEMENTARY_SCHOOL'},
 'HIGHER_EDUCATION': {'codes': ['B14007e17', 'B14007e18', 'B14007e3'],
  'name': 'HIGHER_EDUCATION'},
 'HIGH_SCHOOL': {'codes': ['B14007e13', 'B14007e14', 'B14007e15', 'B14007e16'],
  'name': 'HIGH_SCHOOL'},
 'MIDDLE_SCHOOL': {'codes': ['B14007e10', 'B14007e11', 'B14007e12'],
  'name': 'MIDDLE_SCHOOL'}}